In [268]:
#prepare data 
# %load_ext autoreload
%autoreload 2
import torch
from recall import Recall
from datasets import TripletAudio

K, MAX_CLOSE_NEG, MAX_FAR_NEG = 5, 15, 15
BATCH_SIZE = 128

triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [275]:
# Set up the network and training parameters
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from recall import Recall
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT, RECALL_NUM_CAND = 0.5, 1e-3, 15, 100, 25

#define model
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)

loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

recall = Recall(RECALL_NUM_CAND, K)

initialising model biases
done


In [276]:
#run the model
from trainer import fit
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)

Train: [0/48048 (0%)]	Loss: 5.203338
Train: [12800/48048 (27%)]	Loss: 0.719875
Train: [25600/48048 (53%)]	Loss: 0.459662
Train: [38400/48048 (80%)]	Loss: 0.418596
Epoch: 1/15. Train set: Average loss: 0.5126
Epoch: 1/15. Validation set: Average loss: 0.3728
Train: [0/48048 (0%)]	Loss: 0.306808
Train: [12800/48048 (27%)]	Loss: 0.338364
Train: [25600/48048 (53%)]	Loss: 0.299925
Train: [38400/48048 (80%)]	Loss: 0.279342
Epoch: 2/15. Train set: Average loss: 0.2985
Epoch: 2/15. Validation set: Average loss: 0.2699
Train: [0/48048 (0%)]	Loss: 0.218594
Train: [12800/48048 (27%)]	Loss: 0.257411
Train: [25600/48048 (53%)]	Loss: 0.249688
Train: [38400/48048 (80%)]	Loss: 0.244688
Epoch: 3/15. Train set: Average loss: 0.2495
Epoch: 3/15. Validation set: Average loss: 0.2530
Train: [0/48048 (0%)]	Loss: 0.253314
Train: [12800/48048 (27%)]	Loss: 0.235932
Train: [25600/48048 (53%)]	Loss: 0.234376
Train: [38400/48048 (80%)]	Loss: 0.232255
Epoch: 4/15. Train set: Average loss: 0.2318
Epoch: 4/15. Valid

In [ ]:
%%time
# perform recall on random batch
outputs = []
for d in (triplet_train_dataset, triplet_test_dataset):
    batch_indicies = np.random.choice(list(range(0, d.get_dataset().shape[0])), 500, False)
    queries = d.get_dataset()[batch_indicies]
    true_knns = d.get_knn().iloc[batch_indicies, :]
    outputs.append(recall.calculate(d.get_dataset(), model.embedding_net, queries, true_knns, False))
print(np.mean(outputs[0]), np.mean(outputs[1]))

### Online Selection

In [ ]:
#prepare data 
# %load_ext autoreload
%autoreload 2
import torch
from datasets import AudioTrainDataset, AudioTestDataset
from datasets import BalancedBatchSampler

K = 5
train_dataset = AudioTrainDataset(K)
test_dataset = AudioTestDataset(K)

train_batch_sampler = BalancedBatchSampler(train_dataset)
test_batch_sampler = BalancedBatchSampler(test_dataset)

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler)

In [ ]:
# Set up the network and training parameters
from networks import EmbeddingNet, AnchorNet
from losses import OnlineTripletLoss
from utils import SemihardNegativeTripletSelector, HardestNegativeTripletSelector
from metrics import AverageNonzeroTripletsMetric
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 5, 50

#define model
anchor_net = AnchorNet(train_dataset.data, INPUT_D, OUTPUT_D)
model = EmbeddingNet(anchor_net)
loss_fn = OnlineTripletLoss(MARGIN, SemihardNegativeTripletSelector(MARGIN, train_dataset.KNN))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [ ]:
%autoreload 2
from trainer import fit
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT, metrics=[AverageNonzeroTripletsMetric()])